# MDOF to equivalent SDOF using the first mode only

This IPython notebook converts a pushover curve for an MDOF system into an equivalent SDOF capacity curve, considering the first mode of vibration only. The supplied pushover curve, which is in terms of base shear and roof displacement, is transformed into an equivalent SDOF capacity curve, which is in terms of spectral acceleration and spectral displacement.

For more details on this procedure and the equations used to make the transformation, refer to: Applied Technology Council. (1996). Seismic Evaluation and Retrofit of Concrete Buildings (Vol. 1).

In [1]:
from rmtk.vulnerability.common import utils
from rmtk.vulnerability.mdof_to_sdof.first_mode import first_mode

pushover_file = "../../../../../rmtk_data/capacity_curves_Vb-droof_first_mode.csv"

In [2]:
capacity_curves = utils.read_capacity_curves(pushover_file)
sdof_capacity_curves = first_mode.mdof_to_sdof(capacity_curves)
utils.save_SdSa_capacity_curves(sdof_capacity_curves,'../../../../../rmtk_data/capacity_curves_sdof_first_mode.csv')